In [1]:
from __future__ import division

import sys
sys.path.insert(0, "/home/des01/vitiello/ftaraujo/miniconda3/lib/python3.7/site-packages")
import ROOT

import matplotlib.pyplot as plt

import numpy as np
from numpy import random
from scipy.spatial.distance import pdist, cdist
# from scipy.stats import kstwobign, pearsonr
# from scipy.stats import genextreme

from multiprocessing import Pool

from tqdm import tqdm


Welcome to JupyROOT 6.18/04


In [2]:
def energy(x, y, method='log'):
    x = np.atleast_2d(x).T
    y = np.atleast_2d(y).T
#     print(x.shape)
#     print(y.shape)
#     dx, dy, dxy = pdist(x), pdist(y), cdist(x, y)
    dx, dxy = pdist(x), cdist(x, y)
    n, m = len(x), len(y)
    if method == 'log':
        epsilon = 1e-10
#         dx, dy, dxy = -np.log(dx+epsilon), -np.log(dy+epsilon), -np.log(dxy+epsilon)
        dx, dxy = -np.log(dx+epsilon), -np.log(dxy+epsilon)
    elif method == 'gaussian':
        s = 1
#         dx, dy, dxy = np.exp(-(dx**2)/(2*s**2)), np.exp(-(dy**2)/(2*s**2)), np.exp(-(dxy**2)/(2*s**2))
        dx, dxy = np.exp(-(dx**2)/(2*s**2)), np.exp(-(dxy**2)/(2*s**2))
    elif method == 'linear':
        pass
    else:
        raise ValueError
#     return (-dxy.sum() / (n * m) + dx.sum() / n**2 + dy.sum() / m**2)
    return (-dxy.sum() / (n * m) + dx.sum() / n**2 )


In [3]:
energy([1,2,3], [1,2,3])



-7.598267289890159

In [4]:
def roodataset2numpy(data, n_entries):
    arr = np.empty(n_entries)
#     print(arr.shape)
    for i in range(n_entries):
#         print(data.get(i))
        arr[i] = data.get(i).getRealValue("x")
    return arr

def get_sim_energy(sim_data):
    return energy(sim_data[0], sim_data[1])
    
def get_energy(data, model, n_entries):
    # get observed energy
#     data = roodataset2numpy(data, n_entries)
#     %timeit model_data = roodataset2numpy(model.generate(ROOT.RooArgSet(x), 100000), 100000)
    model_size = 100000
    print("\n\n--> Generating model...")
    model_data = roodataset2numpy(model.generate(ROOT.RooArgSet(x), model_size), model_size)
    print("--> Calculating observed energy...")
    observed_energy = energy(data, model_data)
#     %timeit observed_energy = energy(data, model_data)
    print(observed_energy)
    
    # get energy distribution
    n_toys = 1000
    sim_data = []
    print("--> Generating toy datasets...")
    for t in tqdm(range(n_toys)):
        sim_data.append((roodataset2numpy(model.generate(ROOT.RooArgSet(x), n_entries), n_entries), model_data))
    sim_energy = []
    with Pool(40) as p:
        print("--> Calculating model energy distribution...")
#         %timeit sim_energy = p.map(get_sim_energy, range(1))
#         sim_energy = p.map(get_sim_energy, sim_data)
        sim_energy = list(tqdm(p.imap(get_sim_energy, sim_data), total=n_toys))

        
    # get p-value
    print("--> Getting p-value...")
    p = np.sum(sim_energy >= observed_energy)/len(sim_energy)
#     p = 1

    print("--> Done\n\n\n")
    return observed_energy, sim_energy, p

In [5]:
c1 = ROOT.TCanvas()

In [ ]:
x = ROOT.RooRealVar("x","x",-10,10)
p0_ = ROOT.RooRealVar("p0_","v",0.5,0,10)
p1_ = ROOT.RooRealVar("p1_","v",1,0,10)
p2_ = ROOT.RooRealVar("p2_","v",0.5,0,10)
p3_ = ROOT.RooRealVar("p3_","v",1,0,10)
p4_ = ROOT.RooRealVar("p4_","v",0.5,0,10)
 
    
# mean = ROOT.RooRealVar("mean","Mean of Gaussian",-10,10)
# sigma = ROOT.RooRealVar("sigma","Width of Gaussian",3,-10,10)

true_model = ROOT.RooBernstein("true_model","RooBernstein",x,ROOT.RooArgList(p0_,p1_,p2_,p3_))
# true_model = ROOT.RooGaussian("gauss","gauss(x,mean,sigma)",x,mean,sigma)


# mean = ROOT.RooRealVar("mean","mean",0,-10,10)
# sigma = ROOT.RooRealVar("sigma","sigma",2,0.,10)
# sig = ROOT.RooGaussian("sig","signal p.d.f.",x,mean,sigma)

# coef0 = ROOT.RooRealVar("c0","coefficient #0",1.0,-1.,1)
# coef1 = ROOT.RooRealVar("c1","coefficient #1",0.1,-1.,1)
# coef2 = ROOT.RooRealVar("c2","coefficient #2",-0.1,-1.,1)
# bkg = ROOT.RooChebychev("bkg","background p.d.f.",x,ROOT.RooArgList(coef0,coef1,coef2))

# fsig = ROOT.RooRealVar("fsig","signal fraction",0.1,0.,1.)

# # model(x) = fsig*sig(x) + (1-fsig)*bkg(x)
# true_model = ROOT.RooAddPdf("model","model",ROOT.RooArgList(sig,bkg),ROOT.RooArgList(fsig))


################################################################
################################################################
################################################################


p0 = ROOT.RooRealVar("p0","v",1,0,10)
p1 = ROOT.RooRealVar("p1","v",1,0,10)
p2 = ROOT.RooRealVar("p2","v",1,0,100)
p3 = ROOT.RooRealVar("p3","v",1,0,10)
p4 = ROOT.RooRealVar("p4","v",1,0,10)
p5 = ROOT.RooRealVar("p5","v",1,0,10)


n_data = 1000
data = true_model.generate(ROOT.RooArgSet(x),n_data)

xframe = x.frame()
data.plotOn(xframe, ROOT.RooLinkedList())
true_model.plotOn(xframe,ROOT.RooFit.LineColor(ROOT.kGray))
data_array = roodataset2numpy(data, n_data)


# leg1 = ROOT.TLegend(0.65,0.73,0.86,0.87);
leg1 = ROOT.TLegend();
# leg1.SetFillColor(ROOT.kWhite);
# leg1.SetLineColor(ROOT.kWhite);
leg1.AddEntry(data,"Data", "P");
# leg1->AddEntry(“model”,“Signal + background”,“LP”);
# leg1->AddEntry(“background”,“Background only”, “LP”);
# leg1->AddEntry(“signal only”,“Signal only”, “LP”);

print("##################################")
print("--> 0th order:")
p0.setVal(1)
p1.setVal(1)
p2.setVal(1)
p3.setVal(1)
p4.setVal(1)
p5.setVal(1)
pol_0_order = ROOT.RooBernstein("pol_0_order","RooBernstein",x,ROOT.RooArgList(p0))
pol_0_order_fit = pol_0_order.fitTo(data,ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.Save())
nll_0 = pol_0_order_fit.minNll()
p0.setConstant(True)
p1.setConstant(True)
p2.setConstant(True)
p3.setConstant(True)
p4.setConstant(True)
p5.setConstant(True)
en_0 = get_energy(data_array, pol_0_order, n_data)
plt.hist(en_0[1], 30) ;
plt.axvline(x=en_0[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_0[2]))
plt.show()
pol_0_order.plotOn(xframe,ROOT.RooFit.LineColor(2+0), ROOT.RooFit.Name("pol_0_order"))
leg1.AddEntry(xframe.findObject("pol_0_order"),"0th order", "L");

print("p0: "+str(p0.getVal()))
p0.setConstant(False)
p1.setConstant(False)
p2.setConstant(False)
p3.setConstant(False)
p4.setConstant(False)
p5.setConstant(False)

print("##################################")
print("--> 1st order:")
p0.setVal(1)
p1.setVal(1)
p2.setVal(1)
p3.setVal(1)
p4.setVal(1)
p5.setVal(1)
pol_1_order = ROOT.RooBernstein("pol_1_order","RooBernstein",x,ROOT.RooArgList(p0,p1))
pol_1_order_fit = pol_1_order.fitTo(data,ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.Save())
nll_1 = pol_1_order_fit.minNll()
p0.setConstant(True)
p1.setConstant(True)
p2.setConstant(True)
p3.setConstant(True)
p4.setConstant(True)
p5.setConstant(True)
en_1 = get_energy(data_array, pol_1_order, n_data)
plt.hist(en_1[1], 30) ;
plt.axvline(x=en_1[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_1[2]))
plt.show()
pol_1_order.plotOn(xframe,ROOT.RooFit.LineColor(2+1), ROOT.RooFit.Name("pol_1_order"))
leg1.AddEntry(xframe.findObject("pol_1_order"),"1st order", "L");

print("p0: "+str(p0.getVal()))
print("p1: "+str(p1.getVal()))
p0.setConstant(False)
p1.setConstant(False)
p2.setConstant(False)
p3.setConstant(False)
p4.setConstant(False)
p5.setConstant(False)

print("##################################")
print("--> 2nd order:")
p0.setVal(1)
p1.setVal(1)
p2.setVal(1)
p3.setVal(1)
p4.setVal(1)
p5.setVal(1)
pol_2_order = ROOT.RooBernstein("pol_2_order","RooBernstein",x,ROOT.RooArgList(p0,p1,p2))
pol_2_order_fit = pol_2_order.fitTo(data,ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.Save())
nll_2 = pol_2_order_fit.minNll()
p0.setConstant(True)
p1.setConstant(True)
p2.setConstant(True)
p3.setConstant(True)
p4.setConstant(True)
p5.setConstant(True)
en_2 = get_energy(data_array, pol_2_order, n_data)
plt.hist(en_2[1], 30) ;
plt.axvline(x=en_1[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_2[2]))
plt.show()
pol_2_order.plotOn(xframe,ROOT.RooFit.LineColor(2+2), ROOT.RooFit.Name("pol_2_order"))
leg1.AddEntry(xframe.findObject("pol_2_order"),"2nd order", "L");

print("p0: "+str(p0.getVal()))
print("p1: "+str(p1.getVal()))
print("p2: "+str(p2.getVal()))
p0.setConstant(False)
p1.setConstant(False)
p2.setConstant(False)
p3.setConstant(False)
p4.setConstant(False)
p5.setConstant(False)

print("##################################")
print("--> 3rd order:")
p0.setVal(1)
p1.setVal(1)
p2.setVal(1)
p3.setVal(1)
p4.setVal(1)
p5.setVal(1)
pol_3_order = ROOT.RooBernstein("pol_3_order","RooBernstein",x,ROOT.RooArgList(p0,p1,p2,p3))
pol_3_order_fit = pol_3_order.fitTo(data,ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.Save())
nll_3 = pol_3_order_fit.minNll()
p0.setConstant(True)
p1.setConstant(True)
p2.setConstant(True)
p3.setConstant(True)
p4.setConstant(True)
p5.setConstant(True)
en_3 = get_energy(data_array, pol_3_order, n_data)
plt.hist(en_3[1], 30) ;
plt.axvline(x=en_3[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_3[2]))
plt.show()

pol_3_order.plotOn(xframe,ROOT.RooFit.LineColor(2+3), ROOT.RooFit.Name("pol_3_order"))
leg1.AddEntry(xframe.findObject("pol_3_order"),"3rd order", "L");

print("p0: "+str(p0.getVal()))
print("p1: "+str(p1.getVal()))
print("p2: "+str(p2.getVal()))
print("p3: "+str(p3.getVal()))
p0.setConstant(False)
p1.setConstant(False)
p2.setConstant(False)
p3.setConstant(False)
p4.setConstant(False)
p5.setConstant(False)

print("##################################")
print("--> 4th order:")
p0.setVal(1)
p1.setVal(1)
p2.setVal(1)
p3.setVal(1)
p4.setVal(1)
p5.setVal(1)
pol_4_order = ROOT.RooBernstein("pol_4_order","RooBernstein",x,ROOT.RooArgList(p0,p1,p2,p3,p4))
pol_4_order_fit = pol_4_order.fitTo(data,ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.Save())
nll_4 = pol_4_order_fit.minNll()
p0.setConstant(True)
p1.setConstant(True)
p2.setConstant(True)
p3.setConstant(True)
p4.setConstant(True)
p5.setConstant(True)
en_4 = get_energy(data_array, pol_4_order, n_data)
plt.hist(en_4[1], 30) ;
plt.axvline(x=en_4[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_4[2]))
plt.show()
pol_4_order.plotOn(xframe,ROOT.RooFit.LineColor(2+4), ROOT.RooFit.Name("pol_4_order"))
leg1.AddEntry(xframe.findObject("pol_4_order"),"4th order", "L");

print("p0: "+str(p0.getVal()))
print("p1: "+str(p1.getVal()))
print("p2: "+str(p2.getVal()))
print("p3: "+str(p3.getVal()))
print("p4: "+str(p4.getVal()))
p0.setConstant(False)
p1.setConstant(False)
p2.setConstant(False)
p3.setConstant(False)
p4.setConstant(False)
p5.setConstant(False)

print("##################################")
print("--> 5th order:")
p0.setVal(1)
p1.setVal(1)
p2.setVal(1)
p3.setVal(1)
p4.setVal(1)
p5.setVal(1)
pol_5_order = ROOT.RooBernstein("pol_5_order","RooBernstein",x,ROOT.RooArgList(p0,p1,p2,p3,p4,p5))
pol_5_order_fit = pol_5_order.fitTo(data,ROOT.RooFit.PrintLevel(-1), ROOT.RooFit.Save())
nll_5 = pol_5_order_fit.minNll()
p0.setConstant(True)
p1.setConstant(True)
p2.setConstant(True)
p3.setConstant(True)
p4.setConstant(True)
p5.setConstant(True)
en_5 = get_energy(data_array, pol_5_order, n_data)
plt.hist(en_5[1], 30) ;
plt.axvline(x=en_5[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_5[2]))
plt.show()
pol_5_order.plotOn(xframe,ROOT.RooFit.LineColor(2+5), ROOT.RooFit.Name("pol_5_order"))
leg1.AddEntry(xframe.findObject("pol_5_order"),"5th order", "L");

print("p0: "+str(p0.getVal()))
print("p1: "+str(p1.getVal()))
print("p2: "+str(p2.getVal()))
print("p3: "+str(p3.getVal()))
print("p4: "+str(p4.getVal()))
print("p5: "+str(p5.getVal()))
p0.setConstant(False)
p1.setConstant(False)
p2.setConstant(False)
p3.setConstant(False)
p4.setConstant(False)
p5.setConstant(False)


# data.plotOn(xframe, ROOT.RooLinkedList())
xframe.Draw()
leg1.Draw()
c1.Draw()

##################################
--> 0th order:


--> Generating model...
--> Calculating observed energy...



  1%|          | 9/1000 [00:00<00:11, 89.49it/s]

0.7545415045927638
--> Generating toy datasets...



100%|██████████| 1000/1000 [00:11<00:00, 89.98it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

--> Calculating model energy distribution...


In [ ]:
print(en_0[0])
print(en_1[0])
print(en_2[0])
print(en_3[0])
print(en_4[0])

In [ ]:
plt.hist(en_0[1]) ;
plt.axvline(x=en_0[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_0[2]))
plt.show()

plt.hist(en_1[1]) ;
plt.axvline(x=en_1[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_1[2]))
plt.show()

plt.hist(en_2[1]) ;
plt.axvline(x=en_1[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_2[2]))
plt.show()

plt.hist(en_3[1]) ;
plt.axvline(x=en_3[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_3[2]))
plt.show()

plt.hist(en_4[1]) ;
plt.axvline(x=en_4[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_4[2]))
plt.show()

plt.hist(en_5[1]) ;
plt.axvline(x=en_5[0], color="tab:red")
plt.xlabel('energy')
plt.ylabel('Probability')
plt.title('0-order ; p-value: '+str(en_5[2]))
plt.show()

In [22]:
delta_01 = 2*(nll_0 - nll_1)
delta_12 = 2*(nll_1 - nll_2)
delta_23 = 2*(nll_2 - nll_3)
delta_34 = 2*(nll_3 - nll_4)

In [16]:
print(delta_01)
print(delta_12)
print(delta_23)
print(delta_34)

19.128169004932715
0.2548771201127238
11.474907277194689
0.5927764316120374


In [18]:
print("p-value 0-1 = "+str(ROOT.TMath.Prob(delta_01, 1)))
print("p-value 1-2 = "+str(ROOT.TMath.Prob(delta_12, 1)))
print("p-value 2-3 = "+str(ROOT.TMath.Prob(delta_23, 1)))
print("p-value 3-4 = "+str(ROOT.TMath.Prob(delta_34, 1)))

p-value 0-1 = 9.053583767401081e-300
p-value 1-2 = 0.12057039061408968
p-value 2-3 = 4.367861373852782e-56
p-value 3-4 = 0.15152296737866053
